# SQL setup

In [ ]:

import tkinter as tk
from tkinter import messagebox
from tkinter import ttk
import mysql.connector

# MySQL connection parameters
db_config = {
    'host': 'localhost',
    'user': 'anshi',
    'password': '#1Anshianay',
    'database': 'Ehealthcare',
}

# Function to execute SQL queries
def execute_query(query, values=None):
    try:
        conn = mysql.connector.connect(**db_config)
        cursor = conn.cursor()

        if values:
            cursor.execute(query, values)
        else:
            cursor.execute(query)

        conn.commit()
    except mysql.connector.Error as err:
        messagebox.showerror("Database Error", f"Error: {err}")
    finally:
        if conn.is_connected():
            cursor.close()
            conn.close()

# Create the patient_info table
execute_query("""
CREATE TABLE IF NOT EXISTS patient_info (
    id INT PRIMARY KEY AUTO_INCREMENT,
    Email VARCHAR(255) UNIQUE NOT NULL,
    Name VARCHAR(255),
    Contact_no VARCHAR(255),
    Password VARCHAR(255)
)
""")

# Create the doctor_info table
execute_query("""
CREATE TABLE IF NOT EXISTS doctor_info (
    id INT PRIMARY KEY AUTO_INCREMENT,
    Email VARCHAR(255) UNIQUE NOT NULL,
    Name VARCHAR(255),
    Contact_no VARCHAR(255),
    Password VARCHAR(255),
    Qualifications VARCHAR(255),
    Speciality VARCHAR(255)
)
""")

def get_connection():
    try:
        conn = mysql.connector.connect(**db_config)
        return conn
    except mysql.connector.Error as err:
        messagebox.showerror("Database Error", f"Error: {err}")


# Signup and login mechanism

In [2]:
def p_signup(p_mail, p_pass, p_name, p_contact):
    query = "INSERT INTO patient_info (Email, Name, Contact_no, Password) VALUES (%s, %s, %s, %s)"
    values = (p_mail, p_name, p_contact, p_pass)
    execute_query(query, values)

def p_login(p_mail, p_pass):
    query = "SELECT * FROM patient_info WHERE Email = %s AND Password = %s"
    values = (p_mail, p_pass)
    user = fetch_data(query, values)

    if user:
        messagebox.showinfo("Login", "Patient login successful!")
        create_patient_homepage()
    else:
        messagebox.showinfo("Login", "Invalid email or password. Please try again!")

def d_signup(d_mail, d_pass, d_name, d_contact, d_qual, d_speciality):
    query = "INSERT INTO doctor_info (Email, Name, Contact_no, Password, Qualifications, Speciality) VALUES (%s, %s, %s, %s, %s, %s)"
    values = (d_mail, d_name, d_contact, d_pass, d_qual, d_speciality)
    execute_query(query, values)

def d_login(d_mail, d_pass):
    query = "SELECT * FROM doctor_info WHERE Email = %s AND Password = %s"
    values = (d_mail, d_pass)
    user = fetch_data(query, values)

    if user:
        messagebox.showinfo("Login", "Doctor login successful!")
        create_doctor_homepage()
    else:
        messagebox.showinfo("Login", "Invalid email or password. Please try again!")

def fetch_data(query, values=None):
    try:
        conn = get_connection()
        cursor = conn.cursor()

        if values:
            cursor.execute(query, values)
        else:
            cursor.execute(query)

        data = cursor.fetchall()
        return data
    except mysql.connector.Error as err:
        messagebox.showerror("Database Error", f"Error: {err}")
    finally:
        if 'conn' in locals() and conn.is_connected():
            cursor.close()
            conn.close()

            

# Patient profile

In [3]:


def patient_my_profile():
    def save_patient_info():
        # Function to save patient information
        patient_name = name_entry.get()
        patient_age = age_entry.get()
        patient_gender = gender_var.get()
        patient_diagnosis = diagnosis_entry.get("1.0", tk.END)

        save_patient_to_db(patient_name, patient_age, patient_gender, patient_diagnosis)

    def save_patient_to_db(name, age, gender, diagnosis):
        query = "INSERT INTO patient_info (Name, Age, Gender, Diagnosis) VALUES (%s, %s, %s, %s)"
        values = (name, age, gender, diagnosis)
        execute_query(query, values)
        
    def clear_fields():
        # Function to clear input fields
        name_entry.delete(0, tk.END)
        age_entry.delete(0, tk.END)
        gender_var.set("Male")  # Set default gender to Male
        diagnosis_entry.delete("1.0", tk.END)

    # Create a new window
    profile_window = tk.Toplevel()
    profile_window.title("Patient Information Profile")

    # Create and place widgets
    frame = ttk.Frame(profile_window, padding="10")
    frame.grid(column=0, row=0, sticky=(tk.W, tk.E, tk.N, tk.S))

    # Patient Name
    ttk.Label(frame, text="Patient Name:").grid(column=0, row=0, sticky=tk.W)
    name_entry = ttk.Entry(frame, width=30)
    name_entry.grid(column=1, row=0, sticky=tk.W, padx=10, pady=5)

    # Patient Age
    ttk.Label(frame, text="Patient Age:").grid(column=0, row=1, sticky=tk.W)
    age_entry = ttk.Entry(frame, width=10)
    age_entry.grid(column=1, row=1, sticky=tk.W, padx=10, pady=5)

    # Patient Gender
    ttk.Label(frame, text="Patient Gender:").grid(column=0, row=2, sticky=tk.W)
    gender_var = tk.StringVar()
    gender_combobox = ttk.Combobox(frame, textvariable=gender_var, values=["Male", "Female", "Other"])
    gender_combobox.grid(column=1, row=2, sticky=tk.W, padx=10, pady=5)
    gender_combobox.set("Male")  # Set default gender to Male

    # Patient Diagnosis
    ttk.Label(frame, text="Patient Diagnosis:").grid(column=0, row=3, sticky=tk.W)
    diagnosis_entry = tk.Text(frame, wrap=tk.WORD, width=30, height=5)
    diagnosis_entry.grid(column=1, row=3, sticky=tk.W, padx=10, pady=5)

    # Save Button
    save_button = ttk.Button(frame, text="Save", command=save_patient_info)
    save_button.grid(column=0, row=4, columnspan=2, pady=10)

    # Clear Button
    clear_button = ttk.Button(frame, text="Clear", command=clear_fields)
    clear_button.grid(column=0, row=5, columnspan=2, pady=5)
    
    
    

# Homepage

In [4]:
# Global variable for company information
company_info_text = """
Welcome to MedLink - Your Trusted Healthcare Partner!

MedLink is committed to providing exceptional healthcare services. Our platform connects doctors and patients seamlessly, ensuring quality care and a smooth experience.

Explore the features designed to make your healthcare journey efficient and comfortable. For any assistance, feel free to reach out to our support team.

- The MedLink Team
"""

def create_doctor_homepage():
    homepage = tk.Toplevel(root)
    homepage.title("MedLink - Doctor Homepage")
    homepage.geometry(f"{w}x{h}")
    homepage.attributes('-fullscreen', True)

    # Left Panel
    left_panel = tk.Frame(homepage, width=300, bg='lightgray', padx=10, pady=10)
    left_panel.pack(side=tk.LEFT, fill=tk.Y, expand=False)
    left_panel.pack_propagate(False)

    # MedLink Label
    tk.Label(left_panel, text="MedLink", font=('Helvetica', 18, 'bold'), bg='lightgray').pack(pady=30)

    # Doctor Buttons
    tk.Button(left_panel, text="My Profile", command=show_doctor_profile).pack(pady=50)
    tk.Button(left_panel, text="View Appointments", command=show_appointments).pack(pady=50)

    # Right Panel
    right_panel = tk.Frame(homepage, bg='white', padx=20, pady=20)
    right_panel.pack(side=tk.RIGHT, fill=tk.BOTH, expand=True)

    # MedLink Company Information
    tk.Label(right_panel, text=company_info_text, font=('Helvetica', 12)).pack()

    
    
def create_patient_homepage():
    homepage = tk.Toplevel(root)
    homepage.title("MedLink - Patient Homepage")
    homepage.geometry(f"{w}x{h}")
    homepage.attributes('-fullscreen', True)

    # Left Panel
    left_panel = tk.Frame(homepage, width=300,bg='lightgray', padx=10, pady=10)
    left_panel.pack(side=tk.LEFT, fill=tk.Y, expand=False)
    left_panel.pack_propagate(False)

    # MedLink Label
    tk.Label(left_panel, text="MedLink", font=('Helvetica', 18, 'bold'), bg='lightgray').pack(pady=30)

    # Patient Buttons
    tk.Button(left_panel, text="My Profile", command=patient_my_profile).pack(pady=50)
    tk.Button(left_panel, text="View Appointments", command=show_appointments).pack(pady=50)
    tk.Button(left_panel, text="Search Doctors", command=search_doctors).pack(pady=50)
    tk.Button(left_panel, text="Talk to AI Chatbot", command=talk_to_chatbot).pack(pady=50)

    # Right Panel
    right_panel = tk.Frame(homepage, bg='white', padx=20, pady=20)
    right_panel.pack(side=tk.RIGHT, fill=tk.BOTH, expand=True)

    # MedLink Company Information
    tk.Label(right_panel, text=company_info_text, font=('Helvetica', 12)).pack()

def show_doctor_profile():
    messagebox.showinfo("Doctor Profile", "This is the doctor's profile.")

def show_appointments():
    messagebox.showinfo("Appointments", "Viewing Appointments.")

def search_doctors():
    messagebox.showinfo("Search Doctors", "Searching for doctors.")

def talk_to_chatbot():
    messagebox.showinfo("AI Chatbot", "Talking to AI Chatbot.")
    

# Signup window

In [5]:
def create_signup_window(user_type):
    # Close the main window
    root.iconify()

    signup_window = tk.Toplevel(root)
    signup_window.title(f"MedLink - {user_type} Signup")
    signup_window.geometry(f"{w}x{h}")
    signup_window.attributes('-fullscreen', True)

    # Labels
    tk.Label(signup_window, text="MedLink", font=('Helvetica', 23)).pack(pady=20)
    tk.Label(signup_window, text=f'{user_type} Signup', font=('Helvetica', 18)).pack()

    # Entry fields
    tk.Label(signup_window, text="Name: ").pack()
    name_entry = tk.Entry(signup_window, width=30)
    name_entry.pack()

    tk.Label(signup_window, text="Email: ").pack()
    email_entry = tk.Entry(signup_window, width=30)
    email_entry.pack()

    tk.Label(signup_window, text="Password: ").pack()
    password_entry = tk.Entry(signup_window, width=30, show='*')
    password_entry.pack()

    tk.Label(signup_window, text="Contact Number: ").pack()
    contact_entry = tk.Entry(signup_window, width=30)
    contact_entry.pack()

    if user_type == "Doctor":
        tk.Label(signup_window, text="Qualification: ").pack()
        qualification_entry = tk.Entry(signup_window, width=30)
        qualification_entry.pack()

        tk.Label(signup_window, text="Speciality: ").pack()
        speciality_entry = tk.Entry(signup_window, width=30)
        speciality_entry.pack()

    # Signup button function
    def signup():
        user_info = {
            'Name': name_entry.get(),
            'Email': email_entry.get(),
            'Password': password_entry.get(),
            'Contact Number': contact_entry.get(),
        }

        if user_type == "Doctor":
            user_info.update({
                'Qualification': qualification_entry.get(),
                'Speciality': speciality_entry.get()
            })

            # Call doctor signup function
            d_signup(user_info['Email'], user_info['Password'], user_info['Name'],
                     user_info['Contact Number'], user_info['Qualification'], user_info['Speciality'])
        elif user_type == "Patient":
            # Call patient signup function
            p_signup(user_info['Email'], user_info['Password'], user_info['Name'], user_info['Contact Number'])
        messagebox.showinfo("Signup", "Signed up successfully!")
        login()

    # Already a user? Login button function
    def login():
        signup_window.destroy()  # Close the signup window
        
        # Open login window
        login_window = tk.Toplevel(root)
        login_window.title(f"MedLink - {user_type} Login")
        login_window.geometry(f"{w}x{h}")
        login_window.attributes('-fullscreen', True)

        # Labels
        tk.Label(login_window, text="MedLink", font=('Helvetica', 23)).pack(pady=20)
        tk.Label(login_window, text=f'{user_type} Login', font=('Helvetica', 18)).pack()

        # Entry fields
        tk.Label(login_window, text="Email: ").pack()
        email_entry = tk.Entry(login_window, width=30)
        email_entry.pack()

        tk.Label(login_window, text="Password: ").pack()
        password_entry = tk.Entry(login_window, width=30, show='*')
        password_entry.pack()

        # Login button function
        def user_login():
            if user_type == "Doctor":
                # Call doctor login function
                d_login(email_entry.get(), password_entry.get())

            elif user_type == "Patient":
                # Call patient login function
                p_login(email_entry.get(), password_entry.get())

        # Login button
        btn_login = tk.Button(login_window, text='Login', command=user_login, width=23, height=3)
        btn_login.pack(pady=20)

    # Signup button
    btn_signup = tk.Button(signup_window, text='Signup', command=signup, width=23, height=3)
    btn_signup.pack(pady=20)

    # Already a user? Login button
    btn_login = tk.Button(signup_window, text=f'Already a user? Login', command=login)
    btn_login.pack()

# main

In [6]:
# Main window
root = tk.Tk()
root.title("MedLink")
root.attributes('-fullscreen', True)  # Fullscreen mode
root.geometry(f"{root.winfo_screenwidth()}x{root.winfo_screenheight()}")

# Set the window dimensions
w = root.winfo_screenwidth()
h = root.winfo_screenheight()

# Labels
tk.Label(root, text="MedLink", font=('Helvetica', 23)).pack(pady=20)
tk.Label(root, text='Welcome to "MedLink"!', font=('Helvetica', 18)).pack()

# Buttons
btn_doctor = tk.Button(root, text='Doctor Signup', command=lambda: create_signup_window("Doctor"), width=23, height=3)
btn_doctor.pack(pady=20)

btn_patient = tk.Button(root, text='Patient Signup', command=lambda: create_signup_window("Patient"), width=23, height=3)
btn_patient.pack()



root.mainloop()

# ideas

In [ ]:
#connect patient profile page
#make ppf frontened
#make doctor pf
#make appointment page
#connect appointment schedule

#maybe use google calander??


# make appointment

In [9]:
from tkcalendar import DateEntry

def submit_appointment():
    patient_name = name_entry.get()
    selected_doctor = doctor_combobox.get()
    appointment_date = cal.get_date()
    appointment_day = day_combobox.get()
    appointment_time = time_entry.get()
    appointment_status = status_combobox.get()

    # You can add code here to save the appointment details to a database or perform other actions.

    result_label.config(text=f"Appointment made for {patient_name} with {selected_doctor} on {appointment_day}, {appointment_date} at {appointment_time}. Status: {appointment_status}")

# Create the main window
root = tk.Tk()
root.title("Schedule Appointment")

# Set the minimum size of the root window
root.minsize(600, 500)  # You can adjust the values based on your preference

# Create and configure the frame
frame = ttk.Frame(root, padding="50")  # Adjust the padding value to make the frame bigger
frame.grid(row=0, column=0, sticky=(tk.W, tk.E, tk.N, tk.S))
frame.columnconfigure(0, weight=1)
frame.columnconfigure(1, weight=1)

# Create labels and entry widgets
name_label = ttk.Label(frame, text="Patient Name:")
name_label.grid(row=0, column=0, sticky=tk.W, pady=5)

name_entry = ttk.Entry(frame, width=30)
name_entry.grid(row=0, column=1, sticky=tk.W, pady=10)  # Increased pady value for spacing

doctor_label = ttk.Label(frame, text="Select Doctor:")
doctor_label.grid(row=1, column=0, sticky=tk.W, pady=5)

# Sample list of doctors, replace it with your own list
doctors = ["Dr. A", "Dr. B", "Dr. C"]
doctor_combobox = ttk.Combobox(frame, values=doctors, state="readonly")
doctor_combobox.grid(row=1, column=1, sticky=tk.W, pady=10)  # Increased pady value for spacing

date_label = ttk.Label(frame, text="Appointment Date:")
date_label.grid(row=2, column=0, sticky=tk.W, pady=5)

cal = DateEntry(frame, width=30, background='darkblue', foreground='white', borderwidth=2)
cal.grid(row=2, column=1, sticky=tk.W, pady=10)  # Increased pady value for spacing

day_label = ttk.Label(frame, text="Select Day:")
day_label.grid(row=3, column=0, sticky=tk.W, pady=5)

# Sample list of days, replace it with your own list
days = ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday"]
day_combobox = ttk.Combobox(frame, values=days, state="readonly")
day_combobox.grid(row=3, column=1, sticky=tk.W, pady=10)  # Increased pady value for spacing

time_label = ttk.Label(frame, text="Appointment Time:")
time_label.grid(row=4, column=0, sticky=tk.W, pady=5)

time_entry = ttk.Entry(frame, width=30)
time_entry.grid(row=4, column=1, sticky=tk.W, pady=10)  # Increased pady value for spacing

status_label = ttk.Label(frame, text="Appointment Status:")
status_label.grid(row=5, column=0, sticky=tk.W, pady=5)

# Sample list of appointment statuses, replace it with your own list
statuses = ["Confirm", "Cancel"]
status_combobox = ttk.Combobox(frame, values=statuses, state="readonly")
status_combobox.grid(row=5, column=1, sticky=tk.W, pady=10)  # Increased pady value for spacing

# Create a button to submit the appointment
submit_button = ttk.Button(frame, text="Submit Appointment", command=submit_appointment)
submit_button.grid(row=6, column=0, columnspan=2, pady=20)  # Adjust the pady value to provide more space

# Create a label to display the result
result_label = ttk.Label(frame, text="")
result_label.grid(row=7, column=0, columnspan=2, pady=20)  # Adjust the pady value to provide more space

# Start the Tkinter event loop
root.mainloop()